# SET UP

In [2]:
from google.colab import drive
drive.mount('/content/drive')

import sys
sys.path.append('/content/drive/MyDrive/KTL/Olympic KTL/ktl')
sys.path.append('/content/drive/MyDrive/KTL/Olympic KTL/ktl')

Mounted at /content/drive


# THƯ VIỆN

In [3]:
from data import get_data, transform
from envs import TechnicalEnv
from compare import compare, get_benchmark, get_mean_variance
from indicator import precompute_technical_indicators
from visualize import test_model, multitest_analysis, analyze_results, calculate_metrics

from stable_baselines3 import PPO

import numpy as np
import pandas as pd
import json
import os

import warnings
warnings.filterwarnings('ignore')

Phiên bản Vnstock 3.2.1 đã có mặt, vui lòng cập nhật với câu lệnh : `pip install vnstock3 --upgrade`.
Lịch sử phiên bản: https://vnstocks.com/docs/tai-lieu/lich-su-phien-ban
Phiên bản hiện tại 3.1.0

# DỮ LIỆU

In [4]:
df = get_data(start='2018-08-03')
df = transform(df, component='all', fill=True)
close = df['close']
volume = df['volume']

2025-04-29 06:25:57 - vnstock3.common.vnstock - INFO - Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
INFO:vnstock3.common.vnstock:Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
2025-04-29 06:25:57 - vnstock3.common.data.data_explorer - INFO - Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
INFO:vnstock3.common.data.data_explorer:Mã chứng khoán không được chỉ định, chương trình mặc định sử dụng VN30F1M
2025-04-29 06:25:57 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [5]:
train = close['2020-01-01':'2023-01-01']
test = close['2024-01-01':'2025-04-18']

In [6]:
indicator_train = precompute_technical_indicators(close, volume, train)
indicator_test = precompute_technical_indicators(close, volume, test)

# MÔI TRƯỜNG

In [7]:
env_kwarg = {
    'n_assets': 5,
    'initial_assets': None,
    'initial_balance': 100_000_000,
    'reweight': 10,
    'min_pctweight': 0.05,
    'limit': 0,
    'rebalance_window': 30,
    'num_assets_change': 2,
    'transaction_cost': 0.0025
}

In [8]:
train_env = TechnicalEnv(train, indicator_train, **env_kwarg)
test_env = TechnicalEnv(test, indicator_test, **env_kwarg)

# HUẤN LUYỆN

In [9]:
model = PPO.load('/content/drive/MyDrive/KTL/Olympic KTL/model/tech_50_000.zip')

In [10]:
no = test_model(model, test_env, episodes=1000, cash=False)


📋 Kết nối tài khoản Google Drive để lưu các thiết lập của dự án.
Dữ liệu phiên làm việc với Colab của bạn sẽ bị xóa nếu không lưu trữ vào Google Drive.

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#ha = pd.DataFrame(no)
#ha.to_csv('/content/drive/MyDrive/KTL/Olympic KTL/result/tech.csv', index=False)

# CHECKPOINT

## Function

In [ ]:
def checkpoint_test_model(model, env, csv_path='results.csv',
                          total_episodes=10000, checkpoint_size=10, cash=False):
    """
    Chạy test_model theo từng đợt checkpoint_size episode,
    ghi kết quả vào csv_path cho đến khi đủ total_episodes thì dừng.
    """
    existing_episodes = 0
    if os.path.exists(csv_path) and os.path.getsize(csv_path) > 0:
        old_df = pd.read_csv(csv_path)
        if not old_df.empty:
            existing_episodes = old_df["episode"].max() + 1

    print(f"Đã có sẵn {existing_episodes} episode trong '{csv_path}'.")

    while existing_episodes < total_episodes:
        episodes_to_run = min(checkpoint_size, total_episodes - existing_episodes)
        new_results = test_model(model, env, episodes=episodes_to_run, cash=cash)

        for i, result in enumerate(new_results):
            result['episode'] = existing_episodes + i

        df = pd.DataFrame(new_results)

        json_cols = ['portfolio_values', 'weight_history', 'asset_history',
                     'rebalance_history', 'reweight_history', 'metrics']
        for col in json_cols:
            if col in df.columns:
                df[col] = df[col].apply(json.dumps)

        file_exists = os.path.exists(csv_path) and os.path.getsize(csv_path) > 0
        df.to_csv(csv_path, mode='a' if file_exists else 'w', index=False, header=not file_exists)

        existing_episodes += episodes_to_run
        print(f"Checkpoint: Đã hoàn thành {existing_episodes}/{total_episodes} episode.")

    print("Hoàn tất! Đã đủ số episode yêu cầu.")

## Run

In [ ]:
checkpoint_test_model(model, test_env, csv_path='/content/drive/MyDrive/KTL/Olympic KTL/result/tech.csv',
                      total_episodes=5000, checkpoint_size=10, cash=False)

# CONVERT

## Function

In [ ]:
def convert_test_result_to_dict(csv_path='/content/drive/MyDrive/KTL/Olympic KTL/result/tech.csv'):
    """Chuyển đổi DataFrame test_result sang danh sách từ điển, lọc theo model cụ thể."""
    import ast
    test_result = pd.read_csv(csv_path)
    result_list = []
    for _, row in test_result.iterrows():
        episode_data = {
            "episode": int(row["episode"]),
            "portfolio_values": ast.literal_eval(row["portfolio_values"]),
            "weight_history": ast.literal_eval(row["weight_history"]),
            "asset_history": ast.literal_eval(row["asset_history"]),
            "rebalance_history": ast.literal_eval(row["rebalance_history"]),
            "reweight_history": ast.literal_eval(row["reweight_history"]),
            "metrics": ast.literal_eval(row["metrics"])
        }
        result_list.append(episode_data)

    return result_list

## Run

In [ ]:
test_result = convert_test_result_to_dict()

In [ ]:
import pickle
import gzip

# Lưu dict với nén Gzip
def save_dict(data, filename):
    with gzip.open(filename, 'wb') as f:
        pickle.dump(data, f, protocol=pickle.HIGHEST_PROTOCOL)

# Đọc dict từ file nén
def load_dict(filename):
    with gzip.open(filename, 'rb') as f:
        return pickle.load(f)

In [ ]:
loaded_dict = load_dict('/content/drive/MyDrive/KTL/Olympic KTL/result/tech.pkl.gz')

# PHÂN TÍCH

In [11]:
def compare(rl_results, mean_var_df, vn_index_df=None, how='sharpe'):
    """
    So sánh kết quả của agent RL với phương pháp Mean Variance Optimization và VN-Index.

    Parameters:
    -----------
    rl_results : list
        Kết quả từ hàm test_model, chứa thông tin về hiệu suất của agent RL
    mean_var_df : pandas.DataFrame
        DataFrame chứa giá trị portfolio của phương pháp Mean Variance Optimization,
        với index là datetime và cột 'Mean Var' chứa giá trị portfolio
    vn_index_df : pandas.DataFrame, optional
        DataFrame chứa dữ liệu OHLCV của VN-Index, với index là datetime
    how : str, default='sharpe'
        Tiêu chí để chọn episode tốt nhất:
        'return': tìm episode có tổng lợi nhuận cao nhất
        'risk': tìm episode có biến động (volatility) thấp nhất
        'sharpe': tìm episode có Sharpe Ratio cao nhất
        'random': chọn ngẫu nhiên một episode
        'all': hiển thị tất cả các tiêu chí trên cùng với giá trị trung bình của tất cả episode

    Returns:
    --------
    dict
        Dictionary chứa các biểu đồ so sánh và phân tích số liệu
    """
    import numpy as np
    import pandas as pd
    import plotly.express as px
    import plotly.graph_objects as go
    from plotly.subplots import make_subplots
    import datetime
    import random

    # Nếu how='all', sẽ hiển thị tất cả các tiêu chí
    if how == 'all':
        # Tìm các chỉ số episode tốt nhất theo từng tiêu chí
        max_return_idx = np.argmax([r['metrics']['total_return'] for r in rl_results])
        min_risk_idx = np.argmin([r['metrics']['volatility'] for r in rl_results])
        max_sharpe_idx = np.argmax([r['metrics']['sharpe_ratio'] for r in rl_results])

        # Lấy kết quả từ các episode tốt nhất
        max_return_result = rl_results[max_return_idx]
        min_risk_result = rl_results[min_risk_idx]
        max_sharpe_result = rl_results[max_sharpe_idx]

        print(f"Selected episodes: Max Return {max_return_idx}, Min Risk {min_risk_idx}, Max Sharpe {max_sharpe_idx}")

        # Sử dụng episode có Sharpe cao nhất cho các metrics chính
        best_rl_result = max_sharpe_result
        best_episode_idx = max_sharpe_idx
        criterion = 'Multiple Criteria'
    else:
        # Chọn episode dựa trên tiêu chí
        if how == 'return':
            best_episode_idx = np.argmax([r['metrics']['total_return'] for r in rl_results])
            criterion = 'Total Return'
        elif how == 'risk':
            best_episode_idx = np.argmin([r['metrics']['volatility'] for r in rl_results])
            criterion = 'Lowest Volatility'
        elif how == 'sharpe':
            best_episode_idx = np.argmax([r['metrics']['sharpe_ratio'] for r in rl_results])
            criterion = 'Sharpe Ratio'
        elif how == 'random':
            best_episode_idx = random.randint(0, len(rl_results) - 1)
            criterion = 'Random Selection'
        else:
            raise ValueError("Parameter 'how' must be one of: 'return', 'risk', 'sharpe', 'random', 'all'")

        print(f"Selected episode {best_episode_idx} based on {criterion}")
        best_rl_result = rl_results[best_episode_idx]

    # Lấy metrics từ episode tốt nhất
    rl_metrics = best_rl_result['metrics']

    # Đảm bảo dữ liệu mean_var_df có index là datetime
    if not isinstance(mean_var_df.index[0], (datetime.datetime, np.datetime64, pd.Timestamp)):
        mean_var_df.index = pd.date_range(start='2020-01-01', periods=len(mean_var_df))

    # Chuẩn bị dữ liệu cho Mean Variance
    mean_var_returns = mean_var_df['Mean Var'].pct_change().dropna()

    # Tính toán các metrics cho Mean Variance
    portfolio_values = mean_var_df['Mean Var'].values
    mean_var_metrics = calculate_metrics(portfolio_values)

    # Chuẩn bị dữ liệu cho VN-Index nếu có
    vn_index_metrics = None
    if vn_index_df is not None:
        # Đảm bảo VN-Index có index là datetime
        if not isinstance(vn_index_df.index[0], (datetime.datetime, np.datetime64, pd.Timestamp)):
            vn_index_df.index = pd.date_range(start='2020-01-01', periods=len(vn_index_df))

        # Lấy giá đóng cửa (close) từ dữ liệu OHLCV
        if 'close' in vn_index_df.columns:
            vn_index_series = vn_index_df['close']
        else:
            # Sử dụng cột đầu tiên nếu không có cột 'close'
            vn_index_series = vn_index_df.iloc[:, 0]

        # Tính toán các metrics cho VN-Index (sử dụng cùng khoảng thời gian với mean_var_df)
        # Lấy dữ liệu VN-Index cho cùng khoảng thời gian với mean_var_df
        common_dates = mean_var_df.index.intersection(vn_index_df.index)
        if len(common_dates) > 0:
            vn_index_aligned = vn_index_series.loc[common_dates].values
            vn_index_metrics = calculate_metrics(vn_index_aligned)
            # Chuẩn bị dữ liệu returns cho các phép tính sau này
            vn_index_returns = pd.Series(vn_index_aligned).pct_change().dropna()

    # Tạo DataFrame để so sánh
    comparison_data = {
        'Metric': ['Total Return', 'Annual Return', 'Volatility', 'Sharpe Ratio', 'Max Drawdown'],
    }

    # Thêm metrics cho các chiến lược khác nhau
    if how == 'all':
        # Thêm metrics cho từng chiến lược RL
        comparison_data['RL Max Return'] = [
            max_return_result['metrics']['total_return'],
            max_return_result['metrics']['annual_return'],
            max_return_result['metrics']['volatility'],
            max_return_result['metrics']['sharpe_ratio'],
            max_return_result['metrics']['max_drawdown']
        ]

        comparison_data['RL Min Risk'] = [
            min_risk_result['metrics']['total_return'],
            min_risk_result['metrics']['annual_return'],
            min_risk_result['metrics']['volatility'],
            min_risk_result['metrics']['sharpe_ratio'],
            min_risk_result['metrics']['max_drawdown']
        ]

        comparison_data['RL Max Sharpe'] = [
            max_sharpe_result['metrics']['total_return'],
            max_sharpe_result['metrics']['annual_return'],
            max_sharpe_result['metrics']['volatility'],
            max_sharpe_result['metrics']['sharpe_ratio'],
            max_sharpe_result['metrics']['max_drawdown']
        ]

        # Tính metrics trung bình của tất cả các episode nếu cần
        avg_metrics = {
            'total_return': np.mean([r['metrics']['total_return'] for r in rl_results]),
            'annual_return': np.mean([r['metrics']['annual_return'] for r in rl_results]),
            'volatility': np.mean([r['metrics']['volatility'] for r in rl_results]),
            'sharpe_ratio': np.mean([r['metrics']['sharpe_ratio'] for r in rl_results]),
            'max_drawdown': np.mean([r['metrics']['max_drawdown'] for r in rl_results])
        }

        comparison_data['RL Average'] = [
            avg_metrics['total_return'],
            avg_metrics['annual_return'],
            avg_metrics['volatility'],
            avg_metrics['sharpe_ratio'],
            avg_metrics['max_drawdown']
        ]
    else:
        # Trường hợp thông thường chỉ có một RL Agent
        comparison_data['RL Agent'] = [
            rl_metrics['total_return'],
            rl_metrics['annual_return'],
            rl_metrics['volatility'],
            rl_metrics['sharpe_ratio'],
            rl_metrics['max_drawdown']
        ]

    # Thêm Mean Variance
    comparison_data['Mean Variance'] = [
        mean_var_metrics['total_return'],
        mean_var_metrics['annual_return'],
        mean_var_metrics['volatility'],
        mean_var_metrics['sharpe_ratio'],
        mean_var_metrics['max_drawdown']
    ]

    # Thêm VN-Index nếu có
    if vn_index_metrics:
        comparison_data['VN-Index'] = [
            vn_index_metrics['total_return'],
            vn_index_metrics['annual_return'],
            vn_index_metrics['volatility'],
            vn_index_metrics['sharpe_ratio'],
            vn_index_metrics['max_drawdown']
        ]

    comparison_df = pd.DataFrame(comparison_data)

    # Chuẩn bị dữ liệu cho biểu đồ so sánh giá trị danh mục
    rl_values = best_rl_result['portfolio_values']

    # Xác định khoảng thời gian chung
    min_length = min(len(rl_values), len(mean_var_df))
    dates = mean_var_df.index[:min_length]

    # Chuẩn bị DataFrame cho biểu đồ
    performance_data = {
        'Date': dates,
    }

    # Thêm dữ liệu cho trường hợp 'all'
    if how == 'all':
        # Thêm giá trị của các episode tốt nhất
        # Chuẩn hóa giá trị ban đầu về 1 để dễ so sánh
        performance_data['RL Max Return'] = np.array(max_return_result['portfolio_values'][:min_length]) / max_return_result['portfolio_values'][0]
        performance_data['RL Min Risk'] = np.array(min_risk_result['portfolio_values'][:min_length]) / min_risk_result['portfolio_values'][0]
        performance_data['RL Max Sharpe'] = np.array(max_sharpe_result['portfolio_values'][:min_length]) / max_sharpe_result['portfolio_values'][0]

        # Tính giá trị trung bình của tất cả các episode
        all_normalized_values = []
        for result in rl_results:
            if len(result['portfolio_values']) >= min_length:
                normalized_values = np.array(result['portfolio_values'][:min_length]) / result['portfolio_values'][0]
                all_normalized_values.append(normalized_values)

        # Tính trung bình nếu có đủ dữ liệu
        if all_normalized_values:
            all_normalized_values = np.array(all_normalized_values)
            performance_data['RL Average'] = np.mean(all_normalized_values, axis=0)
    else:
        # Trường hợp thông thường, chỉ hiển thị một đường RL Agent
        performance_data['RL Agent'] = np.array(rl_values[:min_length]) / rl_values[0]

    # Thêm Mean Variance
    performance_data['Mean Variance'] = mean_var_df['Mean Var'].values[:min_length] / mean_var_df['Mean Var'].values[0]

    # Thêm VN-Index nếu có, đảm bảo khớp với khoảng thời gian
    if vn_index_df is not None and len(common_dates) > 0:
        # Cắt data để chỉ giữ lại những ngày khớp với mean_var_df
        common_dates_in_range = [d for d in dates if d in common_dates]

        if len(common_dates_in_range) > 0:
            # Lấy giá trị VN-Index cho các ngày khớp
            vn_index_values = vn_index_series.loc[common_dates_in_range].values

            # Chuẩn hóa giá trị
            if len(vn_index_values) > 0:
                normalized_vn_index = vn_index_values / vn_index_values[0]

                # Tạo một Series mới với index là tất cả các ngày trong dates
                temp_series = pd.Series(index=dates)

                # Gán giá trị chuẩn hóa cho những ngày khớp
                for i, date in enumerate(common_dates_in_range):
                    temp_series.loc[date] = normalized_vn_index[i]

                # Thêm vào performance_data
                # Chỉ sử dụng những giá trị không phải NaN
                valid_indices = ~temp_series.isna()
                if valid_indices.any():
                    performance_data['VN-Index'] = temp_series.fillna(method='ffill').values

    performance_df = pd.DataFrame(performance_data)

    # Tạo biểu đồ hiệu suất so sánh
    # Định nghĩa màu sắc rõ ràng cho các đường
    colors = {
        'RL Agent': '#2e7cee',  # Xanh dương
        'RL Max Return': '#FF4500',  # Đỏ cam
        'RL Min Risk': '#008000',  # Xanh lá
        'RL Max Sharpe': '#9932CC',  # Tím
        'RL Average': '#FFD700',  # Vàng
        'Mean Variance': '#fc6955',  # Hồng
        'VN-Index': '#00CC96'  # Lục
    }

    # Tạo biểu đồ với màu sắc tùy chỉnh
    fig1 = go.Figure()

    # Thêm từng dòng dữ liệu vào biểu đồ với màu sắc tương ứng
    for col in performance_df.columns:
        if col != 'Date':
            fig1.add_trace(go.Scatter(
                x=performance_df['Date'],
                y=performance_df[col],
                mode='lines',
                name=col,
                line=dict(color=colors.get(col, '#000000'), width=2)  # Màu mặc định đen nếu không tìm thấy
            ))

    # Cập nhật layout
    title_suffix = ''
    if how == 'all':
        title_suffix = ': Multiple RL Strategies vs Benchmarks'
    else:
        title_suffix = f': RL Agent ({criterion}) vs Benchmarks'

    fig1.update_layout(
        title=f'So sánh hiệu suất{title_suffix}',
        xaxis_title='Ngày',
        yaxis_title='Giá trị danh mục (chuẩn hóa)',
        template='plotly_white',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    # Tạo biểu đồ so sánh các metrics
    fig3 = px.bar(
        comparison_df,
        x='Metric',
        y=[col for col in comparison_df.columns if col != 'Metric'],
        barmode='group',
        title=f'So sánh các chỉ số hiệu suất',
        labels={'value': 'Giá trị', 'variable': 'Phương pháp'},
        template='plotly_white',
        color_discrete_map={
            'RL Agent': colors['RL Agent'],
            'RL Max Return': colors['RL Max Return'],
            'RL Min Risk': colors['RL Min Risk'],
            'RL Max Sharpe': colors['RL Max Sharpe'],
            'RL Average': colors['RL Average'],
            'Mean Variance': colors['Mean Variance'],
            'VN-Index': colors['VN-Index']
        }
    )

    # Tính drawdowns
    # RL Agent - sử dụng episode được chọn theo tiêu chí
    rl_returns = pd.Series(best_rl_result['portfolio_values']).pct_change().dropna()
    rl_cum_returns = (1 + rl_returns).cumprod()
    rl_rolling_max = rl_cum_returns.expanding().max()
    rl_drawdowns = (rl_cum_returns / rl_rolling_max - 1) * 100

    # Mean Variance
    mean_var_cum_returns = (1 + mean_var_returns).cumprod()
    mean_var_rolling_max = mean_var_cum_returns.expanding().max()
    mean_var_drawdowns = (mean_var_cum_returns / mean_var_rolling_max - 1) * 100

    # Chuẩn bị dữ liệu drawdown
    min_drawdown_length = min(len(rl_drawdowns), len(mean_var_drawdowns), min_length-1)
    drawdown_data = {
        'Date': dates[1:min_drawdown_length+1],
    }

    # Thêm dữ liệu drawdown cho trường hợp 'all'
    if how == 'all':
        # Tính drawdown cho các episode tốt nhất
        for label, result in [
            ('RL Max Return', max_return_result),
            ('RL Min Risk', min_risk_result),
            ('RL Max Sharpe', max_sharpe_result)
        ]:
            returns = pd.Series(result['portfolio_values']).pct_change().dropna()
            cum_returns = (1 + returns).cumprod()
            rolling_max = cum_returns.expanding().max()
            drawdowns = (cum_returns / rolling_max - 1) * 100
            if len(drawdowns) >= min_drawdown_length:
                drawdown_data[label] = drawdowns.values[:min_drawdown_length]

        # Tính drawdown trung bình nếu cần
        if 'RL Average' in performance_df.columns:
            avg_returns = performance_df['RL Average'].pct_change().dropna()
            avg_cum_returns = (1 + avg_returns).cumprod()
            avg_rolling_max = avg_cum_returns.expanding().max()
            avg_drawdowns = (avg_cum_returns / avg_rolling_max - 1) * 100
            if len(avg_drawdowns) >= min_drawdown_length:
                drawdown_data['RL Average'] = avg_drawdowns.values[:min_drawdown_length]
    else:
        # Trường hợp thông thường
        drawdown_data['RL Agent'] = rl_drawdowns.values[:min_drawdown_length]

    # Thêm Mean Variance
    drawdown_data['Mean Variance'] = mean_var_drawdowns.values[:min_drawdown_length]

    # Thêm VN-Index drawdown nếu có
    if vn_index_df is not None and 'VN-Index' in performance_df.columns:
        # Tính drawdown cho VN-Index sử dụng dữ liệu đã căn chỉnh
        try:
            vn_index_aligned_returns = performance_df['VN-Index'].pct_change().dropna()
            vn_cum_returns = (1 + vn_index_aligned_returns).cumprod()
            vn_rolling_max = vn_cum_returns.expanding().max()
            vn_drawdowns = (vn_cum_returns / vn_rolling_max - 1) * 100

            # Đảm bảo độ dài tương thích
            if len(vn_drawdowns) >= min_drawdown_length:
                drawdown_data['VN-Index'] = vn_drawdowns.values[:min_drawdown_length]
        except Exception as e:
            print(f"Không thể tính drawdown cho VN-Index: {e}")

    drawdown_df = pd.DataFrame(drawdown_data)

    # Tạo biểu đồ drawdown với màu sắc tùy chỉnh
    fig4 = go.Figure()

    # Thêm từng dòng dữ liệu vào biểu đồ với màu sắc tương ứng
    for col in drawdown_df.columns:
        if col != 'Date':
            fig4.add_trace(go.Scatter(
                x=drawdown_df['Date'],
                y=drawdown_df[col],
                mode='lines',
                name=col,
                line=dict(color=colors.get(col, '#000000'), width=2)  # Màu mặc định đen nếu không tìm thấy
            ))

    # Cập nhật layout
    drawdown_title_suffix = ' (Multiple RL Strategies)' if how == 'all' else ''
    fig4.update_layout(
        title=f'So sánh Drawdown (%){drawdown_title_suffix}',
        xaxis_title='Ngày',
        yaxis_title='Drawdown (%)',
        template='plotly_white',
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="right",
            x=1
        )
    )

    # Tạo biểu đồ phân phối lợi nhuận
    # Chuẩn bị dữ liệu
    rl_daily_returns = rl_returns * 100  # Chuyển thành phần trăm
    mean_var_daily_returns = mean_var_returns * 100  # Chuyển thành phần trăm

    # Tính số lượng subplot cần thiết và tên các subplot
    subplot_titles = []
    num_columns = 3  # Mỗi hàng có 3 cột
    num_rows = 2     # Chia thành 2 hàng

    if how == 'all':
        # Thêm các tiêu đề cho trường hợp 'all'
        subplot_titles = ['RL Max Return', 'RL Min Risk', 'RL Max Sharpe', 'RL Average', 'Mean Variance']
    else:
        # Trường hợp thông thường
        subplot_titles = ['RL Agent', 'Mean Variance']

    # Thêm VN-Index nếu có
    if vn_index_df is not None and 'VN-Index' in performance_df.columns:
        subplot_titles.append('VN-Index')

    # Tạo subplot grid 2x3 (6 ô)
    fig5 = make_subplots(rows=num_rows, cols=num_columns, subplot_titles=subplot_titles)

    # Thêm histogram cho từng đường dữ liệu
    subplot_idx = 0

    if how == 'all':
        # Thêm histogram cho từng loại episode
        for label, result in [
            ('RL Max Return', max_return_result),
            ('RL Min Risk', min_risk_result),
            ('RL Max Sharpe', max_sharpe_result)
        ]:
            returns = pd.Series(result['portfolio_values']).pct_change().dropna() * 100
            row_idx = subplot_idx // num_columns + 1
            col_idx = subplot_idx % num_columns + 1

            fig5.add_trace(
                go.Histogram(
                    x=returns,
                    name=label,
                    marker_color=colors[label],
                    opacity=0.7,
                    nbinsx=30
                ),
                row=row_idx, col=col_idx
            )
            subplot_idx += 1

        # Thêm histogram cho RL Average nếu có
        if 'RL Average' in performance_df.columns:
            avg_returns = performance_df['RL Average'].pct_change().dropna() * 100
            row_idx = subplot_idx // num_columns + 1
            col_idx = subplot_idx % num_columns + 1

            fig5.add_trace(
                go.Histogram(
                    x=avg_returns,
                    name='RL Average',
                    marker_color=colors['RL Average'],
                    opacity=0.7,
                    nbinsx=30
                ),
                row=row_idx, col=col_idx
            )
            subplot_idx += 1
    else:
        # Trường hợp thông thường, thêm histogram cho RL Agent
        row_idx = subplot_idx // num_columns + 1
        col_idx = subplot_idx % num_columns + 1

        fig5.add_trace(
            go.Histogram(
                x=rl_daily_returns,
                name='RL Agent',
                marker_color=colors['RL Agent'],
                opacity=0.7,
                nbinsx=30
            ),
            row=row_idx, col=col_idx
        )
        subplot_idx += 1

    # Thêm histogram cho Mean Variance
    row_idx = subplot_idx // num_columns + 1
    col_idx = subplot_idx % num_columns + 1

    fig5.add_trace(
        go.Histogram(
            x=mean_var_daily_returns,
            name='Mean Variance',
            marker_color=colors['Mean Variance'],
            opacity=0.7,
            nbinsx=30
        ),
        row=row_idx, col=col_idx
    )
    subplot_idx += 1

    # Thêm histogram cho VN-Index nếu có
    if vn_index_df is not None and 'VN-Index' in performance_df.columns:
        vn_daily_returns = performance_df['VN-Index'].pct_change().dropna() * 100
        if len(vn_daily_returns) > 0:
            row_idx = subplot_idx // num_columns + 1
            col_idx = subplot_idx % num_columns + 1

            fig5.add_trace(
                go.Histogram(
                    x=vn_daily_returns,
                    name='VN-Index',
                    marker_color=colors['VN-Index'],
                    opacity=0.7,
                    nbinsx=30
                ),
                row=row_idx, col=col_idx
            )

    fig5.update_layout(
        title_text=f'Phân phối lợi nhuận hàng ngày (%)',
        template='plotly_white',
        showlegend=False,
        height=600,  # Tăng chiều cao cho phù hợp với grid 2x3
        width=900    # Điều chỉnh chiều rộng phù hợp
    )

    # Tóm tắt kết quả so sánh
    comparison_summary = {
        'performance_plot': fig1,
        'metrics_comparison_plot': fig3,
        'drawdown_comparison_plot': fig4,
        'return_distribution_plot': fig5,
        'metrics_table': comparison_df,
        'best_episode_number': best_episode_idx,
        'selection_criterion': criterion
    }

    return comparison_summary

## Multi Episodes

In [12]:
multitest_analysis(no, test.index)


=== Multi-Episode Analysis ===
Number of episodes: 1000
Mean Return: 10.78%
Mean Risk (Volatility): 22.80%
Mean Sharpe Ratio: 0.36
Mean Maximum Drawdown: -21.62%

Distribution Statistics:
Return Std Dev: 8.79%
Risk Std Dev: 0.89%


## Chart

In [18]:
a = np.random.randint(0,len(no)-1)
print(a)
analyze_results(no[434], test.index)

942


# COMPARE

In [16]:
mvo = get_mean_variance(train, test, env_kwarg['initial_balance'])
vn_index = get_benchmark(start='2018-08-03', market='VN30')

2025-04-29 06:41:41 - vnstock3.common.data.data_explorer - WARNING - Thông tin niêm yết & giao dịch sẽ được truy xuất từ TCBS


In [17]:
comparison_results = compare(no, mvo, vn_index, how='return')
comparison_results['performance_plot'].show()
comparison_results['metrics_comparison_plot'].show()
comparison_results['drawdown_comparison_plot'].show()
comparison_results['return_distribution_plot'].show()

Selected episode 434 based on Total Return
